In [ ]:
pip install transformers

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def computeppl(sen):
    with torch.no_grad():
        model = BertForMaskedLM.from_pretrained('hfl/chinese-bert-wwm-ext')
        model.eval()
        # Load pre-trained model tokenizer (vocabulary)
        tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')
        sentence = sen
        tokenize_input = tokenizer.tokenize(sentence)
        tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
        sen_len = len(tokenize_input)
        sentence_loss = 0.

        for i, word in enumerate(tokenize_input):
            # add mask to i-th character of the sentence
            tokenize_input[i] = '[MASK]'
            mask_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])

            output = model(mask_input)

            prediction_scores = output[0]
            softmax = nn.Softmax(dim=0)
            ps = softmax(prediction_scores[0, i]).log()
            word_loss = ps[tensor_input[0, i]]
            sentence_loss += word_loss.item()

            tokenize_input[i] = word
        ppl = np.exp(-sentence_loss/sen_len)
    return(ppl)

In [ ]:
import re
def getText(fileName):
    all_text = ""
    file_object = open(fileName, 'r',encoding = 'utf-8')
    try:
        all_text += file_object.read()
    finally:
        file_object.close()
    print(all_text)
    re_patten = '[^\u4e00-\u9fa5？。，！]'
    all_text = re.sub(re_patten, "", all_text.strip())
    #reChinese = re.compile('/[(\u4e000-\u9fff)(A-Za-z0-9)(，,（）【】{}！\-!)]+/g ') 
    #all_text = reChinese.findall(all_text) 
    #all_text = all_text.split()
    #for item in all_text:
        #print(item)
    return all_text


In [ ]:
a=getText('/content/drive/MyDrive/英文文档NLP/data.txt')

In [ ]:
path='/content/drive/MyDrive/英文文档NLP/data.txt'

In [ ]:
f = open(path,"r",encoding='utf-8')
a=[]
text = f.read().split()
for line in text:
    if line == '“'or line == '”'or line == '：'or line == '-' or line == '"' or line == '？'or line == '、' or line == '）'or line == '（' or line == '！':
        continue
    else:
        a.append(line)

In [ ]:
a

In [ ]:
ppltotal=0
count=0
for i in a[0:10000]:  
    print(i)
    ppltotal+=computeppl(i)
    count+=1
    print(count)
    print(ppltotal)

In [ ]:
ppl=ppltotal/count

In [ ]:
ppl